In [3]:
import pandas as pd
from datetime import datetime as dt, timedelta
import teradata
import pyodbc
import warnings

In [4]:
#Lee el archivo de los proveedores
def leer_archivos():
    
    #Especificar la ulr para agarrar todos los archivos de ahi, de momento no tengo la url de una compartida entonces por el momento no importa
    #url = ""
    nombre_cts = "CTS 13 Julio 25.xlsx"
    nombre_int = "Inteneg 13 Julio 25.xlsx"
    
    url_cts = nombre_cts
    url_int = nombre_int
    
    dtype = {
        'Cuenta' : str,
        'Tarjeta' : str,
        'Autorizacion1' : str
    }

    #Lee los archivos tal cual como vienen en el sharepoint
    df_cts = pd.read_excel(url_cts, dtype=dtype, parse_dates=["Fcompra1"])
    df_int = pd.read_excel(url_int, dtype = dtype, parse_dates=["Fcompra1"])
    
    df_cts["Fcompra1"] = df_cts["Fcompra1"].dt.strftime("%d/%m/%Y")
    df_int["Fcompra1"] = df_int["Fcompra1"].dt.strftime("%d/%m/%Y")
    
    
    #Esto junta los dos df en uno solo pero no se si es necesario tenerlos separados o si si se puede tenerlos juntos
    df_prove = pd.concat([df_cts,df_int])
    
    return df_prove

def quitar_duplicados(df):

    #Crea la llave
    df["Llave"] = df["Cuenta"] + df["Autorizacion1"] + df["Monto1"].astype(str) + df["Fcompra1"].astype(str) + df["Txn1"].astype(str)
    
    #Borramos los duplicados
    duplicados = df[df.duplicated(subset="Llave", keep="first")].copy()
    duplicados["Errores"] = "Transacción duplicada"
    df = df.drop_duplicates(subset = 'Llave')

    
    
    return df, duplicados
    
def abrir_db():
#Conexion a la base de datos
    server = "MP-VW-DB-004"
    database = "master"


    conn_str = (
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={server};"
        f"DATABASE={database};"
        f"Trusted_Connection=yes;"
    )

    try:
        connection=pyodbc.connect(conn_str)
        print("\n\tConexion exitosa")
    except Exception as e:
            print("Error en la conexion: ",e)
            
    return connection
                    
def ejecutar_query(query,connection):

    print("\n\tHaciendo consulta SQL")
    #Creamos el objeto cursor para poder ejecutar consultas
    cursor = connection.cursor()

    # Realizar las consultas y el resultado lo guardamos en otro dataframe
    df = pd.read_sql(query, connection)
    print("\n\tConsulta realizada")     
    
    return df
    
def cerrar_db(connection):
    # Cerrar la conexión
    connection.close()
    
    return connection

def formato_SQL(df,columna):
    
    #Convertir columnas en formato SQL (con comillas y coma) para la consulta SQL ejemplo 123456 va a ser igual a '123456',
    lista_formato_SQL = df[columna].tolist()
    lista_formato_SQL= ', '.join(f"'{tarjeta}'" for tarjeta in lista_formato_SQL)
    
    return lista_formato_SQL


def query_arqa(cuentas):

    query = f"""/****** Script for SelectTopNRows command from SSMS  ******/
    SELECT CONCAT(Numero_cuenta,floor(Plan_de_Venta)) as Llave,
           [Numero_Cuenta]
          ,[Tipo_Plan]
          ,[Secuencia]
          ,[Store]
          ,[Plan_de_Venta]
          ,[Status_Plan]
          ,[Fecha_Apertura_Plan]
          ,[Importe]
          ,[ORG]
          ,[LOGO]
          ,[Fecha_Carga]
      FROM [VISION_FILES].[dbo].[Reporte_ARQA]
      Where Numero_Cuenta in ({cuentas})

    Order by Fecha_Apertura_Plan desc
    
    """
    
    return query

def teradata_sebas(consulta_por):

    query = f"""SELECT  
    substr(NUM_CAR_CARF,4,16) as PAN
    ,NUM_CTA_CTAF as CTA 
    ,VAL_TRANS_COM as IMPORTE
    ,DAT_TRANS_COM as FECHA_TXN
    ,NOM_ESTB_FPRO as NOMBRE_COMERCIO
    ,NUM_REF_COM as REF_TXN
    ,NOM_FANT_CARF as NOMBRE_CLIENTE
    ,substr(X.VAL_DADO_ORIG_LKP,4,5) AS PLAN 
    ,NUM_AUT_COM
    ,DAT_PROX_CORTE_FAT_ATRF AS Prox_Corte
    ,COD_TIPO_TRANSACAO_FPRO
    
    FROM CARTAO_FINANCIAMENTO_V	
    
    INNER JOIN CONTA_FINANCIAMENTO_V
    ON ID_CTA_CARF = ID_CTA_CTAF
    
    LEFT OUTER JOIN COMPRA_V
    ON ID_CAR_COM = ID_CAR_CARF
    
    INNER JOIN PLANO_CREDITO_V X
    ON ID_PLANO_CRED_COM = X.ID_PLANO_CRED_LKP
    
    INNER JOIN TIPO_TRANSACAO_V
    ON ID_TIPO_TRANS_COM = ID_TIPO_TRANS_LKP
    
    LEFT OUTER JOIN FAT_TRANSACAO_PROSA_V
    ON ID_CAR_FPRO = ID_CAR_COM
    AND NUM_REF_FPRO = NUM_REF_COM
     
    inner join ATRIBUTO_CONTA_FINANCIAMENTO_V
    on ID_CTA_CTAF = ID_CTA_ATRF and current_date between DAT_DE_ATRF and DAT_ATE_ATRF
     
    WHERE
    {consulta_por}
    --and COD_TIPO_TRANSACAO_FPRO in ('01') 
    and DAT_TRANS_COM >= CURRENT_DATE - INTERVAL '60' DAY
    and substr(X.VAL_DADO_ORIG_LKP,4,5) in ('01000','01100','01181','06000')
    """
    return query
    
def conexion_a_teradata():
    
    try:
        print('\n\tIniciando sesión en Teradata...')
    
        host: str = '192.168.17.64'  # Dirección IP o nombre del servidor Teradata
        user: str = 'MPROC_GR'       # Usuario de Teradata
        password: str = 'padrao' # Contraseña de Teradata
        driver: str = "Teradata Database ODBC Driver 20.00"  # Controlador ODBC

        udaExec = teradata.UdaExec(appName="PythonApp", version="1.0", logConsole=False)

        session = udaExec.connect(

            method="odbc",
            system=host,
            username=user,
            password=password,
            driver=driver,
            sslmode='Allow'
        )
        
    except Exception as e:
        print(f"Error en la conexión a Teradata: {str(e)}")
        return None  # Retorna None en caso de error
    
    return session

    
def hacer_consulta(session, query):

    print("\n\tHaciendo consulta Teradata")
    session.execute("DATABASE PM_MstrDB")
    df = pd.read_sql(query, session)
    print("\n\tConsulta realizada")
    return df

def cerrar_conexion(session):
    session.close()

    
def cruces(df,df_teradata,parametro_teradata,parametro_df,corregir_cuentas):
    
    #Crear una llave en terada cuenta&aut&monto&fecha o tarjeta&aut&monto&fecha depende del parametro
    df_teradata["Llave_completa"] = df_teradata[parametro_teradata] + df_teradata["NUM_AUT_COM"] + df_teradata["IMPORTE"].astype(str) + df_teradata["FECHA_TXN"].astype(str) + df_teradata["PLAN"].astype(str)
    df["Llave_completa"] = df[parametro_df] + df["Autorizacion1"] + df["Monto1"].astype(str) + df["Fcompra1"].astype(str) + df["Txn1"].astype(str)
    
    #Busca en las dos consultas de teradata si se encuentran esos casos y asi saber que todos los campos de la llave estan bien
    df_bien = df[df['Llave_completa'].isin(set(df_teradata['Llave_completa']))]
    
    #Cuando las cuentas estan mal (se hizo la consulta por tarjeta y coincidieron casos), asignar la cuenta de teradata
    if corregir_cuentas == True:
        df_bien = pd.merge(df_bien, df_teradata[["Llave_completa", "CTA"]], on="Llave_completa", how="left")
        df_bien["Cuenta"] = df_bien["CTA"]
        df_bien.drop(['CTA'], axis=1, inplace=True)
        
    
    #Los que no tienen algun campo bien, averiguar que campo esta mal y poner el correcto?
    df_mal = df[~df['Llave_completa'].isin(set(df_teradata['Llave_completa']))]
    
    return df_bien, df_mal
    
    
def correccion_datos(df_mal, df_teradata, parametro_teradata, parametro_df):
    
    #Crea la llave corta en teradata cuenta&aut o tarjeta&aut
    df_teradata["Llave"] = df_teradata[parametro_teradata] + df_teradata["NUM_AUT_COM"]
    df_mal["Llave"] = df_mal[parametro_df] + df_mal["Autorizacion1"]
    
    df_no_estan = df_mal[~df_mal["Llave"].isin(set(df_teradata['Llave']))]
    df_mal = df_mal[df_mal["Llave"].isin(set(df_teradata['Llave']))]
    
    df_mal = pd.merge(df_mal, df_teradata[["Llave", "CTA", "IMPORTE", "FECHA_TXN", "PLAN"]], on="Llave", how="left")
    
    df_mal["Cuenta"] = df_mal["CTA"]
    df_mal["Monto1"] = df_mal["IMPORTE"]
    df_mal["Fcompra1"] = df_mal["FECHA_TXN"]
    df_mal["Txn1"] = df_mal["PLAN"]

    df_mal.drop(['IMPORTE', 'FECHA_TXN', 'PLAN', 'CTA'], axis=1, inplace=True)
    
    return df_mal,df_no_estan
    
def consultar_teradata(consulta_por):

    #Hace la consulta en teradata hace la consulta por cuentas, se indica si se consulta por cuenta o por tarjeta
    session = conexion_a_teradata()
    df_teradata = hacer_consulta(session, teradata_sebas(consulta_por))
    cerrar_conexion(session)
   

    #Cambiar el formato de la fecha a dd/mm/aaaa
    df_teradata['FECHA_TXN'] = pd.to_datetime(df_teradata['FECHA_TXN'], dayfirst=True, errors='coerce')
    df_teradata['FECHA_TXN'] = df_teradata['FECHA_TXN'].dt.strftime('%d/%m/%Y')
    df_teradata['IMPORTE'] = df_teradata['IMPORTE'].map(lambda x: ('%.2f' % x).rstrip('0').rstrip('.') if '.' in ('%.2f' % x) else '%.2f' % x)
    df_teradata["PLAN"] = df_teradata["PLAN"].astype(int)
    df_teradata["PAN"] = "000" + df_teradata["PAN"].astype(str)

    df_teradata = df_teradata[~df_teradata["COD_TIPO_TRANSACAO_FPRO"].isin(["21"])]
    
    return df_teradata

def exportar_archivo(df, df_no_estan, df_corregidos,df_errores,df_itim,url):
    with pd.ExcelWriter(url) as writer:
        df.to_excel(writer, "Completo", index=False)
        df_no_estan.to_excel(writer,"No estan en Teradata", index=False)
        df_corregidos.to_excel(writer, "Todos corregidos", index=False)
        df_errores.to_excel(writer, "Errores", index=False)
        df_itim.to_excel(writer, "ITIIM", index=False)
        
        
    print("\n\tArchivo guardado con exito")


def corregir_aut(df, df_teradata,parametro_teradata, parametro_df):

    #Crea la llave sin aut
    df_teradata["Llave"] = df_teradata[parametro_teradata] + df_teradata["IMPORTE"].astype(str) + df_teradata["FECHA_TXN"].astype(str) + df_teradata["PLAN"].astype(str)
    df["Llave"] = df[parametro_df] + df["Monto1"].astype(str) + df["Fcompra1"].astype(str) + df["Txn1"].astype(str)

    df_no_estan = df[~df["Llave"].isin(set(df_teradata['Llave']))]
    df_mal = df[df["Llave"].isin(set(df_teradata['Llave']))]
    
    df_mal = pd.merge(df_mal, df_teradata[["Llave", "CTA", "NUM_AUT_COM"]], on="Llave", how="left")
    
    df_mal["Cuenta"] = df_mal["CTA"]
    df_mal["Autorizacion1"] = df_mal["NUM_AUT_COM"]


    df_mal.drop(['NUM_AUT_COM', 'CTA'], axis=1, inplace=True)
    
    return df_mal,df_no_estan
    
    
def definir_errores(df_mal, df_cuentas_mal, df_cuen_fech_impo_mal,df_aut_mal,df_aut_cuenta_mal,df_estan_historico, df_duplicados):

    df_cuentas_mal["Errores"] = "Cuentas erroneas"
    df_aut_mal["Errores"] = "Autorizador incorrecto"
    df_aut_cuenta_mal["Errores"] = "Cuenta y aut incorrecto"
    df_cuen_fech_impo_mal["Errores"] = "Cuenta erronea ademas de Fecha, monto o plan incorrecto"
    df_mal["Errores"] = "Fecha, monto o plan incorrecto"

    
    df_errores = pd.concat([
        df_mal,
        df_cuentas_mal,
        df_cuen_fech_impo_mal,
        df_aut_mal,
        df_aut_cuenta_mal,
        df_estan_historico,
        df_duplicados
    ], ignore_index=True)

    return df_errores
    
    
def proceso(df):
    
    #Convierte todas las cuentas en formato SQL
    cuentas = formato_SQL(df, "Cuenta")
    tarjetas = formato_SQL(df,"Tarjeta")
    
    #Linea que sirve para complementar el query para saber por que consultar
    consultar_por_cuenta = f"NUM_CTA_CTAF IN ({cuentas})"
    consultar_por_tarjeta = f"NUM_CAR_CARF IN ({tarjetas})"
    
    #Quita los 0 que no nos sirven
    df['Monto1'] = df['Monto1'].map(lambda x: ('%.2f' % x).rstrip('0').rstrip('.') if '.' in ('%.2f' % x) else '%.2f' % x)
    
    #Vamos a hacer la consulta de teradata por cuenta, por lo que el segundo parametro no importa ahorita (no importa la tarjeta por eso los 0000000)
    df_teradata = consultar_teradata(consultar_por_cuenta)
    #Hace los cruces con teradata usando la llave completa, cuenta como uno de los parametros
    #En caso de que no encuentre en teradata, hace los cruces con cuenta&aut y asigna las fehcas y montos
    #Identifica los que aun asi  noestan
    #Se indica que no se van a corregir las cuentas con el false
    df_bien, df_mal = cruces(df, df_teradata, "CTA", "Cuenta", False)
    df_aut_mal,df_no_estan = corregir_aut(df_mal, df_teradata, "CTA", "Cuenta")
    df_mal,df_no_estan = correccion_datos(df_no_estan,df_teradata, "CTA", "Cuenta")
    
    #Hace los cruces con teradata usando la llave completa, tarjeta como uno de los parametros
    #En caso de que no encuentre en teradata, hace los cruces con tarjeta&aut y asigna las fehcas y montos
    #Identifica los que aun asi  noestan
    #Vamos a hacer la consulta de teradata por tarjeta, por lo que el segundo parametro no importa ahorita (no importa la cuenta por eso los 0000000)
    #Se indica que no se van a corregir las cuentas con el false
    df_teradata = consultar_teradata(consultar_por_tarjeta)    
    print("\n\tCargando archivo")
    df_cuentas_mal, df_todo_mal = cruces(df_no_estan, df_teradata, "PAN", "Tarjeta", True)
    df_aut_cuenta_mal,df_cuen_fech_impo_mal = corregir_aut(df_todo_mal,df_teradata, "PAN", "Tarjeta")
    df_cuen_fech_impo_mal,df_no_estan = correccion_datos(df_cuen_fech_impo_mal,df_teradata, "PAN", "Tarjeta")
    
    #Concatenar todos los df corregidos en uno solo (menos el completo y los manuales) 
    df_corregidos = pd.concat([
        df_bien,
        df_mal,
        df_cuentas_mal,
        df_cuen_fech_impo_mal,
        df_aut_mal,
        df_aut_cuenta_mal
    ], ignore_index=True)
    

    df_corregidos, df_duplicados = quitar_duplicados(df_corregidos)
    
    url = "C:\\Users\\MX50086\\Downloads\\Parcializaciones\\casos.xlsx"

    df_errores = pd.DataFrame(columns=["Aquí van a ir los errores"])
    df_itim = pd.DataFrame(columns=["Aquí van a ir el itim"])

    #df_corregidos = asignar_secuencia_tienda(df_corregidos)
    
    #Exporta el archivo con todos los df, se hacen los casos manuales y se agregan a la hoja de "Todos corregidos" del excel, luego compara todos con el historico
    exportar_archivo(df,df_no_estan,df_corregidos,df_errores,df_itim, url)
    
    input("\n\tCORRIJA Y AGREGUE LOS CASOS MANUALES / APRIETE CUALQUIER TECLA PARA CONTINUAR")

    print("\n\tAsignando secuencias")
    df_corregidos = asignar_secuencia_tienda(df_corregidos)
    
    print("\n\tCargando historico")
    df_corregidos, df_estan_historico = comparar_con_historico(df_corregidos)
    
    print("\n\tArmando Hoja Errores y hoja Itiim")
    df_errores = definir_errores(df_mal, df_cuentas_mal, df_cuen_fech_impo_mal,df_aut_mal,df_aut_cuenta_mal,df_estan_historico,df_duplicados)
    
    df_itim = armar_itim(df_corregidos)
    df_inclusiones = armar_inclusiones(df_corregidos)
    df_itim = pd.concat([df_itim,df_inclusiones])

    print("\n\tCargando archivo")
    exportar_archivo(df,df_no_estan,df_corregidos,df_errores,df_itim, url)

    

    
    
def comparar_con_historico(df):
    
    nombre_historico = "Historico.xlsx"
    #Esto es por si no se encuentra en la ruta del codigo
    url_historico = nombre_historico
    
    dtype = {
        "Cuenta" : str,
        "Tarjeta" : str,
        "Autorizacion1": str,
        "PS" : str,
        "Store" : str,
    }
    
    
    #Que el historico sea de una sola hoja, no separada por socios
    df_historico = pd.read_excel(url_historico, dtype=dtype, parse_dates=["Fcompra1"])

    #Poner las columnas fecha e importe con el mismo formato a los del df
    df_historico['Fcompra1'] = pd.to_datetime(df_historico['Fcompra1'], dayfirst=True, errors='coerce')
    df_historico['Fcompra1'] = df_historico['Fcompra1'].dt.strftime('%d/%m/%Y')
    df_historico['Monto1'] = df_historico['Monto1'].map(lambda x: ('%.2f' % x).rstrip('0').rstrip('.') if '.' in ('%.2f' % x) else '%.2f' % x)

    df['Fcompra1'] = pd.to_datetime(df['Fcompra1'], dayfirst=False, errors='coerce')
    df['Fcompra1'] = df['Fcompra1'].dt.strftime('%d/%m/%Y')
    df['Monto1'] = df['Monto1'].astype(float)
    df['Monto1'] = df['Monto1'].map(lambda x: ('%.2f' % x).rstrip('0').rstrip('.') if '.' in ('%.2f' % x) else '%.2f' % x)
    
    
    #Crear llave completa para el historico
    df_historico["Llave_completa"] = df_historico["Cuenta"] + df_historico["Autorizacion1"] + df_historico["Monto1"].astype(str) + df_historico["Fcompra1"].astype(str) + df_historico["Txn1"].astype(str)
    
    #Creamos de nuevo la llave del df debido a que la anterior estaba con los datos incorrectos (Se corrigeron todos los datos menos los de la llave)
    df["Llave_completa"] = df["Cuenta"] + df["Autorizacion1"] + df["Monto1"].astype(str) + df["Fcompra1"].astype(str) + df["Txn1"].astype(str)
    
    
    #Elimina del df los que estan en el historico
    df_estan_historico = df[df["Llave_completa"].isin(set(df_historico['Llave_completa']))]
    df = df[~df["Llave_completa"].isin(set(df_historico['Llave_completa']))]
    
    df_estan_historico["Errores"] = "Ya esta parcializada"
    
    exportar_historico(df,df_historico)
    
    return df, df_estan_historico
    
def exportar_historico(df, df_historico):
    
    nombre_historico = "Historico.xlsx"
    #Esto es por si no se encuentra en la ruta del codigo
    url_historico = nombre_historico
    
    df_hoy = df.drop(['Llave',  'Llave_completa'], axis=1)
    df_historico = df_historico.drop("Llave_completa", axis=1)
    df_historico = pd.concat([df_hoy, df_historico], ignore_index=True)
    
    with pd.ExcelWriter(url_historico) as writer:
        df_historico.to_excel(writer, "Historico", index=False)

    print("\n\tHistorico guardado con exito")
    



def asignar_secuencia_tienda(df):
    
    #Hace la consulta sql por cuenta
    cuentas = formato_SQL(df,"Cuenta")
    
    conn = abrir_db()
    df_sql = ejecutar_query(query_arqa(cuentas), conn)
    cerrar_db(conn)
    
    #Da el formato a los campos importantes
    df_sql['Fecha_Apertura_Plan'] = pd.to_datetime(df_sql['Fecha_Apertura_Plan'], dayfirst=True, errors='coerce')
    df_sql["Plan_de_Venta"] = df_sql["Plan_de_Venta"].astype(int) 
    
    df['Fcompra1'] = pd.to_datetime(df['Fcompra1'], dayfirst=True, errors='coerce')
    
    
    #Crea las llaves para hacer cruces
    df_sql["Llave"] = df_sql["Numero_Cuenta"] + df_sql["Plan_de_Venta"].astype(str)
    df["Llave"] = df["Cuenta"] + df["Txn1"].astype(str)
    
    #Hace los cruces para traerse todas las tiendas y las secuencias de sql de todos los casos
    df_con_secuencia = pd.merge(df, df_sql[["Llave", "Secuencia", "Store", "Fecha_Apertura_Plan"]], on="Llave", how="left")
    
    #Elimina las que sean despues de la fecha de compra
    df_con_secuencia = df_con_secuencia[df_con_secuencia['Fecha_Apertura_Plan'] <= df_con_secuencia['Fcompra1']]

    #Ordena de mas reciente a mas antigua
    df_con_secuencia = df_con_secuencia.sort_values(by='Fecha_Apertura_Plan', ascending=False)
    
    #Elimina duplicados dejando el primero para tener el mas reciente
    df_con_secuencia = df_con_secuencia.drop_duplicates(subset = "Llave_completa", keep='first')

    #Hacemos este cruce de nuuevo solo para ver si algun dato esta vacio
    df =pd.merge(df,df_con_secuencia[["Llave_completa", "Secuencia", "Store"]], on="Llave_completa", how="left")

    df = df.rename(columns={'Secuencia': 'PS'})
    
    return df

def armar_itim(df):

    df_itim = pd.DataFrame()
    df_itim["Cuenta"] = df["Cuenta"]
    df_itim["Parcialidad"] = "0"
    df_itim["Fecha"] =  df["Fcompra1"]
    df_itim["Monto de la compra"] = df["Monto1"]
    df_itim["Plan"] = df["Txn1"]
    df_itim["Secuencia"] = df["PS"]
    df_itim["Tienda"] = df["Store"]
    df_itim[["Transaccion", "Descripcion"]] = df["TipoParcializacion"].map({
    "PF": ("823", "AJUSTE POR PROMOCION PAGOS FIJOS        "),
    "MSI": ("821", "AJUSTE PROMOCION SIN INTERES RED        ")
    }).apply(pd.Series)

    return df_itim

def definir_plan_pf(parcialidad):

    caso = {
        "3": "3103",
        "6": "3116",
        "9": "3104"
    }

    return caso[parcialidad]

def definir_plan_msi(parcialidad):

    caso = {
        "4": "3173",
        "6": "3176",
        "9": "3179",
        "12": "3170",
    }

    return caso[parcialidad]


def armar_itim_inclusiones(df, tipo_parci):
    
    hoy = dt.now()
    hoy = str(hoy.strftime("%d/%m/%Y"))

    df_inclusiones = pd.DataFrame()
    
    df_inclusiones["Cuenta"] = df["Cuenta"]
    df_inclusiones["Parcialidad"] = df["Pagos"]
    df_inclusiones["Fecha"] = hoy
    df_inclusiones["Monto de la compra"] = df["Monto1"]
    df_inclusiones["Secuencia"] = "0"
    df_inclusiones["Tienda"] = df["Store"]
    if tipo_parci == "PF":
        plan = []
        for index,row in df.iterrows():
            plan.append(definir_plan_pf(str(row["Pagos"])))
        df_inclusiones["Plan"] = plan
        df_inclusiones["Transaccion"] = "822"
        df_inclusiones["Descripcion"] = "INC PROMOCION PAGOS FIJOS               "
    else:
        plan = []
        for index,row in df.iterrows():
            plan.append(definir_plan_msi(str(row["Pagos"])))
        df_inclusiones["Plan"] = plan
        df_inclusiones["Transaccion"] = "820"
        df_inclusiones["Descripcion"] = "INC PROMOCION MSI                       "

    return df_inclusiones

def armar_inclusiones(df):

    df["Cuenta_parci"] = df["Cuenta"].astype(str) + df["Pagos"].astype(str)
    df["Monto1"] = df["Monto1"].astype(float)
    df = df.groupby("Cuenta_parci", as_index=False).agg({
    "Monto1": "sum",
    "Cuenta": "first",
    "Pagos": "first",
    "Txn1": "first",
    "Store": "first",
    "TipoParcializacion": "first"
    })

    
    #Separarlas por df o MSI
    df_pf = df[df["TipoParcializacion"] == "PF"].copy()
    df_msi = df[df["TipoParcializacion"] == "MSI"].copy()

    df_pf_inclusiones = armar_itim_inclusiones(df_pf, "PF")
    df_msi_inclusiones = armar_itim_inclusiones(df_msi, "MSI")
    
    #Concatenar los dos df
    df_inclusiones = pd.concat([df_pf_inclusiones,df_msi_inclusiones])

    return df_inclusiones
    
    
    
def main():
    
    
    #Para no hacer todo dos veces en caso de qu los df se deban mantener separados, se puede crear una funcion en donde se ponga 
    #todos los pasos y de parametro reciba un df, finalmente en el main llamamos a esa funcio dos veces una por cada df
    warnings.filterwarnings('ignore')
    df_prove = leer_archivos()
    proceso(df_prove)
    
    #print(df_bien)
    
    
    
    
main()

 


	Iniciando sesión en Teradata...

	Haciendo consulta Teradata

	Consulta realizada

	Iniciando sesión en Teradata...

	Haciendo consulta Teradata

	Consulta realizada

	Cargando archivo

	Archivo guardado con exito



	CORRIJA Y AGREGUE LOS CASOS MANUALES / APRIETE CUALQUIER TECLA PARA CONTINUAR 



	Asignando secuencias

	Conexion exitosa

	Haciendo consulta SQL

	Consulta realizada

	Cargando historico

	Historico guardado con exito

	Armando Hoja Errores y hoja Itiim

	Cargando archivo

	Archivo guardado con exito
